In [3]:
import os

os.chdir("..")
from src.utils import *

data_dir = 'data'
dataset_name = 'vox1_test_segments'

clean_dt_path = os.path.join(data_dir, dataset_name)

clean_dataset = scan_directory_voxceleb2(clean_dt_path)

In [4]:
clean_dataset

,path,person_id,utterance_filename
0,data\vox1_test_segments\id10270\00001_seg_0.wav,id10270,00001_seg_0.wav
1,data\vox1_test_segments\id10270\00001_seg_1.wav,id10270,00001_seg_1.wav
2,data\vox1_test_segments\id10270\00001_seg_2.wav,id10270,00001_seg_2.wav
3,data\vox1_test_segments\id10270\00001_seg_3.wav,id10270,00001_seg_3.wav
4,data\vox1_test_segments\id10270\00001_seg_4.wav,id10270,00001_seg_4.wav
...,...,...,...
9114,data\vox1_test_segments\id10309\00016_seg_2.wav,id10309,00016_seg_2.wav
9115,data\vox1_test_segments\id10309\00017_seg_0.wav,id10309,00017_seg_0.wav
9116,data\vox1_test_segments\id10309\00018_seg_0.wav,id10309,00018_seg_0.wav
9117,data\vox1_test_segments\id10309\00019_seg_0.wav,id10309,00019_seg_0.wav


In [9]:
# method for adding noise to the clean dataset with a given SNR using the noise dataset, with the same structure as the clean dataset

def add_noise_to_dataset(clean_dataset, data_dir, dataset_name, noisy_subdir='noisy', type_of_gen_noise='gaussian', snr=0):
    output_dir_name = f'{dataset_name}_snr_{snr}_noisy_{type_of_gen_noise}'
    output_dir = os.path.join(data_dir, noisy_subdir, type_of_gen_noise, output_dir_name)

    os.makedirs(output_dir, exist_ok=True)

    for index, row in tqdm(clean_dataset.iterrows(), total=clean_dataset.shape[0]):
        clean_path = row['path']
        waveform, sr = torchaudio.load(clean_path)  # load the clean audio
        noisy = add_scaled_noise_snr(waveform.numpy(), snr, type_of_gen_noise)  # add noise to the clean audio
        noisy = torch.tensor(noisy, dtype=torch.float32)
        # make a new directory for the person_id
        person_subdir = os.path.join(output_dir, row['person_id'])
        os.makedirs(person_subdir, exist_ok=True)
        # save the noisy audio in the new directory
        noisy_path = os.path.join(person_subdir, os.path.basename(clean_path))
        torchaudio.save(noisy_path, noisy, sr)



In [10]:

# add noise to the clean dataset with a given SNR using the noise dataset, with the same structure as the clean dataset
snr = 5
type_of_gen_noise = 'gaussian'

add_noise_to_dataset(clean_dataset, data_dir, dataset_name, noisy_subdir='noisy', type_of_gen_noise=type_of_gen_noise, snr=snr)


100%|██████████| 9119/9119 [00:45<00:00, 201.02it/s]


In [13]:
snrs = [0,5,10,15,20]
type_of_gen_noise = "poisson"

for snr in snrs:
    add_noise_to_dataset(clean_dataset, data_dir, dataset_name, noisy_subdir='noisy', type_of_gen_noise=type_of_gen_noise, snr=snr)

100%|██████████| 9119/9119 [00:46<00:00, 197.41it/s]
